In [1]:
# !pip install yfinance

In [48]:
import pandas as pd
import yfinance as yf
import mysql.connector
from mysql.connector import Error
from datetime import datetime, timedelta
import getpass

In [49]:
def get_db_config():
    host = input("Enter the database host (default: localhost): ") or 'localhost'
    user = input("Enter your database username (default: root): ") or 'root'
    password = getpass.getpass("Enter your database password: ")
    database = 'financial_data'

    return {
        'host': host,
        'user': user,
        'password': password,
        'database': database
    }

In [50]:
tickers = {
    'GOLD_FUTURES': 'GC=F',       # Gold Futures 
    'SP_INDEX': '^GSPC',     # S&P 500 Index
    'DJ_INDEX': '^DJI',      # Dow Jones Index
    'EG_CORP': 'EGO',        # Eldorado Gold Corporation
    'EUR_USD': 'EURUSD=X',   # EUR/USD Exchange rate
    'OIL_FUTURES': 'BZ=F',   # Brent Crude Oil Futures
    'WTI_OIL': 'CL=F',       # Crude Oil WTI Futures
    'SILVER_FUTURES': 'SI=F', # Silver Futures
    'US_BOND_RATE': '^TNX',  # US Bond Rate
    'PLATINUM_FUTURES': 'PL=F', # Platinum Futures
    'PALLADIUM_FUTURES': 'PA=F', # Palladium Futures
    'GOLD_MINERS': 'GDX',    # Gold Miners ETF
    'OIL_ETF': 'USO',        # Oil ETF USO
    'USD_INDEX': 'DX-Y.NYB'  # US Dollar Index
}

In [51]:
def connect_to_database(db_config):
    """Connect to the MySQL database."""
    return mysql.connector.connect(**db_config)

def create_database(cursor, db_name):
    """Create a database if it doesn't exist."""
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name};")
    print(f"Database '{db_name}' checked/created.")

def connect_to_financial_database():
    """Connect to the financial database."""
    return mysql.connector.connect(**DB_CONFIG)

In [52]:
def create_table(cursor, table_name):
    """Create a table for the ticker if it doesn't exist."""
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        Date DATE PRIMARY KEY,
        Open DECIMAL(15, 4),
        High DECIMAL(15, 4),
        Low DECIMAL(15, 4),
        Close DECIMAL(15, 4),
        Adj_Close DECIMAL(15, 4),
        Volume BIGINT
    );
    """
    cursor.execute(create_table_query)

In [53]:
def create_views(cursor):
    # 1. Daily Average Price View
    for name in tickers:
        table_name = name.replace(' ', '_')
        view_name = f"{table_name}_daily_avg_price"
        
        cursor.execute(f"""
        CREATE OR REPLACE VIEW {view_name} AS
        SELECT
            Date,
            (Open + Close) / 2 AS Avg_Price
        FROM {table_name};
        """)
        print(f"View '{view_name}' created for daily average price.")

    # 2. Monthly Summary View
    for name in tickers:
        table_name = name.replace(' ', '_')
        view_name = f"{table_name}_monthly_summary"
        
        cursor.execute(f"""
        CREATE OR REPLACE VIEW {view_name} AS
        SELECT
            DATE_FORMAT(Date, '%Y-%m') AS Month,
            AVG(Close) AS Avg_Close,
            MAX(High) AS Max_High,
            MIN(Low) AS Min_Low,
            SUM(Volume) AS Total_Volume
        FROM {table_name}
        GROUP BY DATE_FORMAT(Date, '%Y-%m');
        """)
        print(f"View '{view_name}' created for monthly summary.")

    # 3. Volatility View
    for name in tickers:
        table_name = name.replace(' ', '_')
        view_name = f"{table_name}_volatility"
        
        cursor.execute(f"""
        CREATE OR REPLACE VIEW {view_name} AS
        SELECT
            Date,
            (Close - Open) / Open * 100 AS Daily_Percent_Change
        FROM {table_name};
        """)
        print(f"View '{view_name}' created for daily volatility.")

In [54]:
def create_materialized_views(cursor):
    # 1. Yearly Summary Materialized View
    for name in tickers:
        table_name = name.replace(' ', '_')
        yearly_summary_mv = f"{table_name}_yearly_summary_mv"
        
        cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {yearly_summary_mv} AS
        SELECT
            YEAR(Date) AS Year,
            AVG(Open) AS Avg_Open,
            AVG(Close) AS Avg_Close,
            AVG(High) AS Avg_High,
            AVG(Low) AS Avg_Low
        FROM {table_name}
        GROUP BY YEAR(Date)
        """)
        print(f"Materialized view '{yearly_summary_mv}' created for yearly summaries.")

    # 2. Moving Averages Materialized View (7-day and 30-day moving averages)
    for name in tickers:
        table_name = name.replace(' ', '_')
        moving_avg_mv = f"{table_name}_moving_averages_mv"
        
        cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {moving_avg_mv} AS
        SELECT
            Date,
            Close,
            AVG(Close) OVER (ORDER BY Date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS Moving_Avg_7d,
            AVG(Close) OVER (ORDER BY Date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS Moving_Avg_30d
        FROM {table_name}
        """)
        print(f"Materialized view '{moving_avg_mv}' created for moving averages.")

    # 3. Volume Trend Materialized View
    for name in tickers:
        table_name = name.replace(' ', '_')
        volume_trend_mv = f"{table_name}_volume_trend_mv"
        
        cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {volume_trend_mv} AS
        SELECT
            Date,
            Volume,
            SUM(Volume) OVER (ORDER BY Date) AS Cumulative_Volume
        FROM {table_name}
        """)
        print(f"Materialized view '{volume_trend_mv}' created for volume trends.")

In [55]:
def refresh_materialized_views(cursor):
    # Function to refresh each materialized view by truncating and re-inserting data
    
    # Yearly Summary Materialized View
    for name in tickers:
        table_name = name.replace(' ', '_')
        yearly_summary_mv = f"{table_name}_yearly_summary_mv"
        
        cursor.execute(f"TRUNCATE TABLE {yearly_summary_mv};")
        cursor.execute(f"""
        INSERT INTO {yearly_summary_mv}
        SELECT
            YEAR(Date) AS Year,
            AVG(Open) AS Avg_Open,
            AVG(Close) AS Avg_Close,
            AVG(High) AS Avg_High,
            AVG(Low) AS Avg_Low
        FROM {table_name}
        GROUP BY YEAR(Date)
        """)
        print(f"Materialized view '{yearly_summary_mv}' refreshed for yearly summaries.")

    # Moving Averages Materialized View
    for name in tickers:
        table_name = name.replace(' ', '_')
        moving_avg_mv = f"{table_name}_moving_averages_mv"
        
        cursor.execute(f"TRUNCATE TABLE {moving_avg_mv};")
        cursor.execute(f"""
        INSERT INTO {moving_avg_mv}
        SELECT
            Date,
            Close,
            AVG(Close) OVER (ORDER BY Date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS Moving_Avg_7d,
            AVG(Close) OVER (ORDER BY Date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS Moving_Avg_30d
        FROM {table_name}
        """)
        print(f"Materialized view '{moving_avg_mv}' refreshed for moving averages.")

    # Volume Trend Materialized View
    for name in tickers:
        table_name = name.replace(' ', '_')
        volume_trend_mv = f"{table_name}_volume_trend_mv"
        
        cursor.execute(f"TRUNCATE TABLE {volume_trend_mv};")
        cursor.execute(f"""
        INSERT INTO {volume_trend_mv}
        SELECT
            Date,
            Volume,
            SUM(Volume) OVER (ORDER BY Date) AS Cumulative_Volume
        FROM {table_name}
        """)
        print(f"Materialized view '{volume_trend_mv}' refreshed for volume trends.")

In [56]:
def fetch_and_insert_data():
    # Prompt user for database details
    db_config = get_db_config()

    # Connect to MySQL server (without database)
    connection = connect_to_database({
        'user': db_config['user'],
        'password': db_config['password'],
        'host': db_config['host']
    })

    cursor = connection.cursor()

    try:
        # Check if the database exists and create it if it doesn't
        create_database(cursor, db_config['database'])
        connection.database = db_config['database']  

        for name, ticker in tickers.items():
            print(f"Checking for new data for {name} ({ticker})...")
            
            # Create the table for the ticker
            create_table(cursor, name.replace(' ', '_'))

            # Fetch existing data to find the last date
            cursor.execute(f"SELECT MAX(Date) FROM {name.replace(' ', '_')}")
            last_date = cursor.fetchone()[0]
            
            # Fetch new data from Yahoo Finance
            if last_date is not None:
                last_date = last_date.strftime('%Y-%m-%d')
                start_date = (datetime.strptime(last_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
            else:
                start_date = '2007-07-30'  # Default start date if no data exists

            print(f"Fetching data starting from {start_date}...")
            data = yf.download(ticker, start=start_date, end=datetime.now().strftime('%Y-%m-%d'), interval='1d')

            # Check if data is empty
            if not data.empty:
                # Preprocess data to handle missing values
                data.ffill(inplace=True)  # Forward fill for the entire DataFrame
                if name == 'PLD':
                    data['Close'].bfill(inplace=True)  # Backward fill for Palladium Futures Close column

                # Check for remaining missing values
                missing_values = data.isnull().sum()
                print(f"Missing values after filling for {name}:\n{missing_values}")

                # If there are still missing values, print a warning
                if missing_values.any():
                    print(f"Warning: Missing values still present in {name} after filling.")

                # Insert new data into the database
                for date, row in data.iterrows():
                    values = (
                        date.strftime('%Y-%m-%d'),
                        float(row['Open'].iloc[0]),
                        float(row['High'].iloc[0]),
                        float(row['Low'].iloc[0]),
                        float(row['Close'].iloc[0]),
                        float(row['Adj Close'].iloc[0]),
                        int(row['Volume'].iloc[0])
                    )

                    # SQL insert statement with duplicate key handling
                    sql = f"""
                    INSERT INTO {name.replace(" ", "_")} (Date, Open, High, Low, Close, Adj_Close, Volume)
                    VALUES (%s, %s, %s, %s, %s, %s, %s)
                    ON DUPLICATE KEY UPDATE
                        Open=VALUES(Open), High=VALUES(High), Low=VALUES(Low),
                        Close=VALUES(Close), Adj_Close=VALUES(Adj_Close), Volume=VALUES(Volume);
                    """

                    # Execute insertion
                    cursor.execute(sql, values)

                # Commit changes to the database
                connection.commit()
                print("Data inserted successfully.")

        # Create regular views after all data is inserted
        create_views(cursor)
        print("Regular views created successfully.")

        # Create materialized views after all data is inserted
        create_materialized_views(cursor)
        print("Materialized views created successfully.")

        # Refresh materialized views
        refresh_materialized_views(cursor)
        connection.commit()
        print("Materialized views refreshed successfully.")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        cursor.close()
        connection.close()
        print("MySQL connection closed.")

In [57]:
# def fetch_and_insert_data():
#     # Prompt user for database details
#     db_config = get_db_config()

#     # Connect to MySQL server (without database)
#     connection = connect_to_database({
#         'user': db_config['user'],
#         'password': db_config['password'],
#         'host': db_config['host']
#     })

#     cursor = connection.cursor()

#     try:
#         # Check if the database exists and create it if it doesn't
#         create_database(cursor, db_config['database'])
#         connection.database = db_config['database']  

#         for name, ticker in tickers.items():
#             print(f"Checking for new data for {name} ({ticker})...")
            
#             # Create the table for the ticker
#             create_table(cursor, name.replace(' ', '_'))

#             # Fetch existing data to find the last date
#             cursor.execute(f"SELECT MAX(Date) FROM {name.replace(' ', '_')}")
#             last_date = cursor.fetchone()[0]
            
#             # Fetch new data from Yahoo Finance
#             if last_date is not None:
#                 last_date = last_date.strftime('%Y-%m-%d')
#                 start_date = (datetime.strptime(last_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
#             else:
#                 start_date = '2007-07-30'  # Default start date if no data exists

#             print(f"Fetching data starting from {start_date}...")
#             data = yf.download(ticker, start=start_date, end=datetime.now().strftime('%Y-%m-%d'), interval='1d')

#             if not data.empty:
#                 print(f"Data downloaded for {name}:\n{data}")

#                 # Insert new data into the database
#                 for date, row in data.iterrows():
#                     values = (
#                         date.strftime('%Y-%m-%d'),
#                         float(row['Open'].iloc[0]),
#                         float(row['High'].iloc[0]),
#                         float(row['Low'].iloc[0]),
#                         float(row['Close'].iloc[0]),
#                         float(row['Adj Close'].iloc[0]),
#                         int(row['Volume'].iloc[0])
#                     )

#                     # SQL insert statement with duplicate key handling
#                     sql = f"""
#                     INSERT INTO {name.replace(" ", "_")} (Date, Open, High, Low, Close, Adj_Close, Volume)
#                     VALUES (%s, %s, %s, %s, %s, %s, %s)
#                     ON DUPLICATE KEY UPDATE
#                         Open=VALUES(Open), High=VALUES(High), Low=VALUES(Low),
#                         Close=VALUES(Close), Adj_Close=VALUES(Adj_Close), Volume=VALUES(Volume);
#                     """

#                     # Execute insertion
#                     cursor.execute(sql, values)

#                 # Commit changes to the database
#                 connection.commit()
#                 print("Data inserted successfully.")

#         # Create regular views after all data is inserted
#         create_views(cursor)
#         print("Regular views created successfully.")

#         # Create materialized views after all data is inserted
#         create_materialized_views(cursor)
#         print("Materialized views created successfully.")

#         # Refresh materialized views
#         refresh_materialized_views(cursor)
#         connection.commit()
#         print("Materialized views refreshed successfully.")


#     except mysql.connector.Error as err:
#         print(f"Error: {err}")
#     finally:
#         cursor.close()
#         connection.close()
#         print("MySQL connection closed.")



In [58]:
# def fetch_and_insert_data():
#     # Prompt user for database details
#     db_config = get_db_config()

#     # Connect to MySQL server (without database)
#     connection = connect_to_database({
#         'user': db_config['user'],
#         'password': db_config['password'],
#         'host': db_config['host']
#     })

#     cursor = connection.cursor()

#     try:
#         # Check if the database exists and create it if it doesn't
#         create_database(cursor, db_config['database'])
#         connection.database = db_config['database']  

#         for name, ticker in tickers.items():
#             print(f"Checking for new data for {name} ({ticker})...")
            
#             # Create the table for the ticker
#             create_table(cursor, name.replace(' ', '_'))

#             # Fetch existing data to find the last date
#             cursor.execute(f"SELECT MAX(Date) FROM {name.replace(' ', '_')}")
#             last_date = cursor.fetchone()[0]
            
#             # Fetch new data from Yahoo Finance
#             if last_date is not None:
#                 last_date = last_date.strftime('%Y-%m-%d')
#                 start_date = (datetime.strptime(last_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
#             else:
#                 start_date = '2007-07-30'  # Default start date if no data exists

#             print(f"Fetching data starting from {start_date}...")
#             data = yf.download(ticker, start=start_date, end=datetime.now().strftime('%Y-%m-%d'), interval='1d')

#             if not data.empty:
#                 print(f"Data downloaded for {name}:\n{data}")

#                 # Insert new data into the database
#                 for date, row in data.iterrows():
#                     values = (
#                         date.strftime('%Y-%m-%d'),
#                         float(row['Open'].iloc[0]),
#                         float(row['High'].iloc[0]),
#                         float(row['Low'].iloc[0]),
#                         float(row['Close'].iloc[0]),
#                         float(row['Adj Close'].iloc[0]),
#                         int(row['Volume'].iloc[0])
#                     )

#                     # SQL insert statement with duplicate key handling
#                     sql = f"""
#                     INSERT INTO {name.replace(" ", "_")} (Date, Open, High, Low, Close, Adj_Close, Volume)
#                     VALUES (%s, %s, %s, %s, %s, %s, %s)
#                     ON DUPLICATE KEY UPDATE
#                         Open=VALUES(Open), High=VALUES(High), Low=VALUES(Low),
#                         Close=VALUES(Close), Adj_Close=VALUES(Adj_Close), Volume=VALUES(Volume);
#                     """

#                     # Execute insertion
#                     cursor.execute(sql, values)

#                 # Commit changes to the database
#                 connection.commit()
#                 print("Data inserted successfully.")

#     except mysql.connector.Error as err:
#         print(f"Error: {err}")
#     finally:
#         cursor.close()
#         connection.close()
#         print("MySQL connection closed.")

In [59]:
fetch_and_insert_data()

Enter the database host (default: localhost):  localhost
Enter your database username (default: root):  root
Enter your database password:  ········


[*********************100%***********************]  1 of 1 completed

Database 'financial_data' checked/created.
Checking for new data for GOLD_FUTURES (GC=F)...
Fetching data starting from 2007-07-30...
Missing values after filling for GOLD_FUTURES:
Price      Ticker
Adj Close  GC=F      0
Close      GC=F      0
High       GC=F      0
Low        GC=F      0
Open       GC=F      0
Volume     GC=F      0
dtype: int64


Data inserted successfully.
Checking for new data for SP_INDEX (^GSPC)...
Fetching data starting from 2007-07-30...


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


Missing values after filling for OIL_ETF:
Price      Ticker
Adj Close  USO       0
Close      USO       0
High       USO       0
Low        USO       0
Open       USO       0
Volume     USO       0
dtype: int64


[*********************100%***********************]  1 of 1 completed

Data inserted successfully.
Checking for new data for USD_INDEX (DX-Y.NYB)...
Fetching data starting from 2007-07-30...


Missing values after filling for USD_INDEX:
Price      Ticker  
Adj Close  DX-Y.NYB    0
Close      DX-Y.NYB    0
High       DX-Y.NYB    0
Low        DX-Y.NYB    0
Open       DX-Y.NYB    0
Volume     DX-Y.NYB    0
dtype: int64
Data inserted successfully.
View 'GOLD_FUTURES_daily_avg_price' created for daily average price.
View 'SP_INDEX_daily_avg_price' created for daily average price.
View 'DJ_INDEX_daily_avg_price' created for daily average price.
View 'EG_CORP_daily_avg_price' created for daily average price.
View 'EUR_USD_daily_avg_price' created for daily average price.
View 'OIL_FUTURES_daily_avg_price' created for daily average price.
View 'WTI_OIL_daily_avg_price' created for daily average price.
View 'SILVER_FUTURES_daily_avg_price' created for daily average price.
View 'US_BOND_RATE_daily_avg_price' created for daily average price.
View 'PLATINUM_FUTURES_daily_avg_price' created for daily average price.
View 'PALLADIUM_FUTURES_daily_avg_price' created for daily average price.